In [4]:
import numpy as np
from scipy.optimize import minimize

W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

def linear_method(w, d):
    return ((w/d-1)**2)

def ranking_ratio_method(w, d):
    return (w/d)*np.log(w/d)-(w/d)+1

# def objective(W):
#     return (1/2)*sum(
#         d_k*linear_method(w_k, d_k) for w_k, d_k in zip(W, D)
#     )

def objective(W):
    return sum(
        d_k*ranking_ratio_method(w_k, d_k) for w_k, d_k in zip(W, D)
    )

def constraint(W):
    return A.T@W-B

constraints = {"type" : "eq", "fun" : constraint}

result = minimize(objective, 
                  x0=x0, 
                  method="trust-constr", 
                  constraints=constraints)

In [14]:
class MarginCalibration:

    def __init__(self,
                sampling_probabilities,
                calibration_matrix,
                calibration_target,
                calibration_method):
        self.sampling_probabilities = sampling_probabilities
        self.calibration_matrix = calibration_matrix
        self.calibration_target = calibration_target
        self.calibration_method = calibration_method

    def initialize_sampling_weights(self):
        return np.array([1/prob_i for prob_i in self.sampling_probabilities])

    def _linear_method(w, d):
        return ((w/d-1)**2)

    def _ranking_ratio_method(w, d):
        return (w/d)*np.log(w/d)-(w/d)+1

    def initialize_method(self):
        if self.calibration_method == "linear":
            return self._linear_method
        elif self.calibration_method == "ranking_ratio":
            return self._ranking_ratio_method
        else:
            raise ValueError(f"Invalid value : {self.calibration_method}. Must be one of : 'linear', 'ranking_ratio'")

    def objective(self, calibration_weights):
        
        D = self.initialize_sampling_weights()
        
        return sum(
            d_k*self.initialize_method()(w_k, d_k) for w_k, d_k in zip(calibration_weights, D)
        )

    def constraint(self, calibration_weights):
        return self.calibration_matrix.T@calibration_weights-self.calibration_target

    def calibration(self):
        
        constraints = {"type":"eq", "fun":self.constraint}
        
        x0 = self.initialize_sampling_weights()
        
        return minimize(
            self.objective,
            x0=x0,
            method = "trust-constr",
            constraints = constraints
        )    

In [15]:
W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

MarginCalibration(W_i, A, B, "linear").calibration()

TypeError: MarginCalibration._linear_method() takes 2 positional arguments but 3 were given